# Inspection of DC2 Run 2.2i DR6 Object Table with Dask
### Michael Wood-Vasey (@wmwv)
### Last Verified to Run: 2021-06-14 by MWV

Inspect the Run 2.2i DR6 Object Table  
Using a Dask Cluster on one NERSC node as the backend.

#### Run 2.2i DR6
  * 166 tracts
  * 147 million objects  

Logistics:

1. These tests were conducted on NERSC through the https://jupyter.nersc.gov interface.  
Note: To enable re-rastering when zooming, use the JupyterLab Classic interface.
You can launch this from an active JupyterHub Notebook by selecting "Help->Launch Classic Notebook".
  * You can select the "Running" tab and then select the Notebook you want.
  * You could instead browse through the full filesystem path under the "Files" tab to find your Notebook, but that's a lot more clicking.  You may want to take this aproach to launch some other Notebook that's not currently running under JupyterHub.

2. Requires:
```
dask
dask.distributed
holoviews
datashader
bokeh
pyarrow >= 0.13.1
```

and `graphviz` if you want to see the Dask task graphs.

Up-to-date versions of each of these are available in `desc-python-bleed` kernel

3. This was run using the `desc-python-bleed` kernel

We directly use the DPDD Parquet files.

4. We use Dask, HoloViews, and Datashader to read Parquet files.  For more on each of these, see:

References:  
    https://dask.org  
    https://datashader.org  
    https://parquet.apache.org  
    https://holoviews.org  
    https://holoviz.org
    
In brief:

### Parquet

Parquet is a column-based storage format that's part of a wider Arrow project to provide standardized, high-performance data representations in memory and on disk.  It's commonly used in current data science and large data volume processing, and is the current selected standard for Rubin Observatory LSST Data Management on-disk representations of output data catalogs.  The DESC Data Access Team is thus similarly using Parquet as the default underlying data format for representations of DC2 data as processed by the LSST DM Science Pipelines.

### Dask

Dask allows us to do processing by dividing tasks into individual workers.  These workers allow us to take fuller use of available memory and processors, including those on other machines.

Dask is solving the needs to:

1. Load more data than fit into memory. You can delay this in either time or space.
   * Delaying in time would be if you running on a memory-limited machine, then Dask will be able to chunk through the work units without simultaneously needing the full amount of memory to hold all of the data at once.
   * Delaying in space means spinning up additional machines.  This is often particularly powerful when connecting your front-end machine (e.g., a NERSC JupyterHub job is limited to 42 GB memory), to several full cluster compute nodes (e.g., a NERSC Haswell node is 32 real cores, 128 GB).  

2. Distributing work across multiple processors. Python and numpy/scipy are not naturally parallel or easily parallelizable. One of the common things we will do with large datasets is aggregate for both analysis and visualization. Being able to do this aggregation in parallel is a significant gain.


### HoloViews

HoloViews is Dask aware and can provide Dask the correct information to build a Task Graph that effectively parallelizes the requisite data loading and computation.  HoloViews can use either bokeh or matplotlib backends.  If you directly use the matplotlib backend with a Dask DataFrame it will not appropriately parallelize across the workers and instead do lots of stuff in serial.  Bokeh also gives some nice interactive capabilities and HoloViews knows how to appropriate set up the linking and call backs to enable coordinated zooming and selection.

## Approach.

Our limitation will be available memory and disk I/O.  There's a trade-off between them.  The more times we're willing to load from disk, the less we have to keep in memory.

1. Load enough data to get an index of `good` objects.  
  1. We need 'good' column and 'magerr_{SNRfilter}' column.
  2. Potentially also define `star` and `galaxy` indices.
  3. Persist thes indexes.
2. Go through each sections exploring different columns of the database
  1. RA, Dec
  2. color, color
  3. number density of galaxies
  4. extendedness
  5. blendedness
  6. shape (moments)
  7. PSF

TODO
1. Plot colorbars for data-shaded plots
2. Control order of samples in Overlays.  Overlay currently alphabetizes in the legends.  I would like to learn how to force the order so the match the heights of the contributions, e.g. "good", "galaxy", "star".

## Import Needed Modules

In [ ]:
import math
import os

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

import astropy.units as u
import healpy as hp

In [ ]:
import colorcet

import dask
from dask.distributed import Client

from bokeh.models import HoverTool
import dask.array as da
import dask.dataframe as dd
import datashader as ds
import holoviews as hv
from holoviews.operation import histogram
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.plotting.util import process_cmap
from holoviews.streams import RangeXY

In [ ]:
hv.extension('bokeh')

In [ ]:
cmap = 'viridis'

If you set LOCAL_DASK to True and are running on the shared JupyterHub nodes, then select a more limited set of data, likely one Tract would be good.   
There's a 42 GB limit on memory directly in the JupyterHub environment at NERSC, 
which you will hit pretty quickly if you try to read in all of the data.

If you ask for an exclusive node for your JupyterHub session, then you should be able to run all of the data.

In [ ]:
LOCAL_DASK = True
# LOCAL_DASK = False

## Start our Dask Cluster


For simple testing and illustration of how to use dask, holoview, and datashader here you can run locally on just one tract.
To run on the full set of DR6, you'll need to set up a a set of nodes to support Dask distributed.  Basically you just machine that can hold the data in memory on the reader.

### Start a local Dask Cluster

In [ ]:
if LOCAL_DASK:
    client = Client()

### Start a Dask Cluster on an Interactive Nodes

So instead, in a separate Terminal on Cori, ask for a pair of Nodes from the `interactive` queue.  This generally completes in seconds.  We ask for 2 Nodes because we'd like the full ~256 GB of availablel memory to store the data and the intermediate copies that often get made in some of the plots below:

```
salloc -N 2 -C haswell --qos=interactive -t 04:00:00 -L SCRATCH
```

And then once on the first Node, where you'll get put after the `salloc` complets, load the right Python environment.  We also want to move to the $C
```
python /global/common/software/lsst/common/miniconda/start-kernel-cli.py desc-python-dev
```

And then start up the Dask Cluster
```
NUM_WORKERS=16
SCHEDULER_FILE=${CSCRATCH}/scheduler.json
rm -rf ${SCHEDULER_FILE}
dask-scheduler --scheduler-file ${SCHEDULER_FILE} &
dask-worker --nprocs ${NUM_WORKERS} --scheduler-file ${SCHEDULER_FILE} --local-directory /tmp &
```

Then exit the environment and go to the second Node.

```
python /global/common/software/lsst/common/miniconda/start-kernel-cli.py desc-python-dev
```

```
NUM_WORKERS=16
SCHEDULER_FILE=${CSCRATCH}/scheduler.json
dask-worker --nprocs ${NUM_WORKERS} --scheduler-file ${SCHEDULER_FILE} --local-directory /tmp &
```

The nodes will be printed out when the `salloc` launches.  And if you forget, you can look them up under the `SLURM_NODELIST` environment variable.

We used `CSCRATCH` above instead of `SCRATCH`, when specifying the location of the scheduler file, because `CSCRATCH` will be consistent across the nodes, whereas `SCRATCH` will be empty on the second compute node (the first one gets some special environment passing through because it's an interactive session).

We connect to this Dask cluster through a shared agreement on where the `SCHEDULER_FILE` is.

We then configure the dashboard URL to use the JupyterHub proxy service.
We here set the formatting string template to the correct value.
Once we actually connect the client, then client can then tell us the full link.

In [ ]:
if not LOCAL_DASK:
    scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler.json")
    dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"
    client = Client(scheduler_file=scheduler_file)

In [ ]:
client

In [ ]:
# client.restart()

## Define Catalog and Subsampling

We specify the directory path that contains all of the individual Parquet files.  If we want to read just one tract, we could pass the full path name of just the tract.  If we wanted to specify a set of tracts to run, we could pass in a regex that matches those tracts.  Or we could pass in a list that contained the file paths or directory paths to read from.

In [ ]:
# desc_data_dir = "/global/cfs/cdirs/lsst/shared/"
# desc_data_dir = "/bgfs/mwood-vasey/wmwv"
desc_data_dir = "/Volumes/PS1/DESC"

In [ ]:
data_release = "dr6"

run_data_dir = f"DC2-prod/Run2.2i/dpdd/Run2.2i-{data_release}/object_dpdd_only"
all_tracts_data_path = os.path.join(desc_data_dir, run_data_dir)
sample_tract_data_path = os.path.join(data_path, "object_dpdd_tract4854.parquet")

data_path = sample_tract_data_path

In [ ]:
print(data_path)

## Load Data

In [ ]:
filters = ('u', 'g', 'r', 'i', 'z', 'y')

Parquet is a column-based storage format. If we specify the columns we want to load, then we avoid the memory overhead of having to store the other columns.
Note: In principle Dask should be able to figure out what columns we actually used and only used those.  But it doesn't really know what to keep around.

In [ ]:
# Select good detections:
#  1. Marked as 'good' in catalog flags.
#  2. SNR in given band > threshold
#  3. In defined simulation range
snr_threshold = 5
snr_filter = 'i'

# We want to do a SNR cut, but magerr is the thing already calculated
# So we'll redefine our SNR in terms of magerr
magerr_cut = (2.5 / np.log(10)) / snr_threshold

Now just load the columns we need to define our `good`, `star`, and `galaxy` indexes.

In [ ]:
# columns = ['objectId']
columns = ['objectId', f'magerr_{snr_filter}', 'good', 'extendedness']

In [ ]:
# df = dd.read_parquet(data_path, columns=columns, engine='pyarrow')
df = dd.read_parquet(
    data_path,
    columns=columns,
    engine="pyarrow",
    kwargs={"dataset": {"use_legacy_dataset": False}},
)

Each file is its own tract which has its own block of ObjectIDs
We'd like to use ObjectIDs as the index for future subsetting so we need to set up the index to do that.  But we also need to tell Dask that it won't need to do any re-shuffling across partitions.

Dask doesn't know anything about how to divide things up yet:

In [ ]:
df.npartitions

In [ ]:
df.divisions

A `set_index` takes about a minute if the data are on disk[*]

But, we can use the results of this `set_index` to run `set_index` for free on any future read of columns from this same set of Parquet files.

[*] Re-runs of this Notebook can be faster because the data that we're asking for may still be in the cache of the OS or network file system.

Note that the `set_index` by default removes the "objectId" column as an explicit column because it's now the index.  We could choose to change this behavior it we want to be able to still say df["objectId"], but it's fine for now.  The only place we're really going to use the `objectId` is indeed as an index.

In [ ]:
# Note that because "objectId" gets removed when setting the index, you can't rerun the `set_index` cell.
df = df.set_index("objectId", sorted=True)

Dask now knows the objectId index division points of each partition.  Note that it doesn't actually track the length, but just knowing the divisions is sufficient to look up things by the index by sending them to the right worker.

In [ ]:
df.npartitions

In [ ]:
df.divisions

Calculating the size of the Dask DataFrame is relatively fast because Dask knows to just ask each of the Parquet files how many rows does it have.  It doesn't have to fully load it into memory.

In [ ]:
len_df = len(df)

In [ ]:
print(len_df)

In [ ]:
good = df["good"] & (df[f"magerr_{snr_filter}"] < magerr_cut)
galaxy = good & (df["extendedness"] > 0)
star = good & (df["extendedness"] == 0)

Each of these is now a Dask Series, which has the same index as the original DataFrame, and a boolean column.  Note that they haven't been computed yet.

In [ ]:
good

In [ ]:
good.npartitions

### Visualize the Task Graph

We need `graphviz` to visualize the Task Graph.

We can then look at the structure of the computations.  You can see the read steps at the bottom, and then a `getitem` for each column we're getting: "good", "magerr_snrfilter", and "extendedness" (in left-to-right order in the task graph).

The work is easily divided across the 166 partitions.

In [ ]:
star.visualize()

## Persist the data in Dask Cluster Worker memory

Dask actively purges data from memory when its no longer needed by the Dask Task Graph currently doing the computation.

This `good` index will be used everytime we load a new set of columns below so let's persist it.  If you're running the `dask.distributed` scheduler[*], executing this cell doesn't immediately do anything more than return a Dask `Future` object that contains the underlying task graph.  When that task graph is executed, the results will be saved in distributed memory once their computed.

[*] Running the `dask.distributed` scheduler is common, even on a single node.  Read more at
https://docs.dask.org/en/latest/scheduling.html

In [ ]:
good = good.persist()
star = star.persist()
galaxy = galaxy.persist()

The persistence question will come up again as we load the additional columns.  Each plot below is its own separate computation.  Dask has no way of knowing that it's going to use those data again in the next plot.  We thus explicitly tell Dask to persist this data frame so that it's still already loaded when we plot the next thing.

If you don't have the physical memory across your Dask installation (whether local or remote), then skip this persist step.  Running each of the plots will require re-reading the data and be a bit slower than if we had memory to keep all of the data, but will work.

The DF length calculation is quick because `len` gets remapped in useful ways in a Dask DataFrame.

In [ ]:
print(f'Total: {len(df)}')


But to get the number of good, star, and galaxy objects, we need to use `sum` instead of `len`.  These Series have an entry for each row that is set to True or False.  So their length isn't the thing we need.  Instead it's the `sum` of the values.

But `sum(df)` does not get handled like `len(df)`.  It doesn't resolve and call a dunder function, it instead gets applied serially (switching between threads across workers) and take almost a minute.

So we instead need to remember to call `star.sum().compute()` instead of `sum(compute)`.

In [ ]:
print(
    f"Total: {len(df)}, Good: {good.sum().compute()}, Stars: {star.sum().compute()}, Galaxies: {galaxy.sum().compute()}"
)

At this point the Dask Dashboard reports that I'm at 3.7 GB of "managed memory"
https://distributed.dask.org/en/latest/worker.html?highlight=unmanaged#using-the-dashboard-to-monitor-memory-usage

In [ ]:
good

In [ ]:
good.dask.dependents

In [ ]:
df.dask.dependents

In [ ]:
del df

The index should use about 1 byte * 147 million entries = 147 MB, but instead I get 1.3 GB when calling `memory_usage`, which says it reports the memory usage in bytes:

In [ ]:
good.memory_usage().compute()

Note that `good` hasn't actually been computed yet:

In [ ]:
good

## Object Density in RA, Dec

1. Load 'objectId', 'RA', 'Dec'.
2. Set index using _already-known_ divisions from our `good` index.
3. Use `good` index to only show those objects.

In [ ]:
ra_dec_df = dd.read_parquet(
    data_path,
    columns=["objectId", "ra", "dec"],
    engine="pyarrow",
    kwargs={"dataset": {"use_legacy_dataset": False}},
)

Using the already-calculated `divisions` value from `good` and specifying that things are already sorted makes this `set_index` operation free.  *It is critically important to use `sorted` and `divisions`* to avoid expensive shuffle operations.

In [ ]:
ra_dec_df = ra_dec_df.set_index("objectId", sorted=True, divisions=good.divisions)

DC2 Run 2.x WFD and DDF regions
https://docs.google.com/document/d/18nNVImxGioQ3tcLFMRr67G_jpOzCIOdar9bjqChueQg/view
https://github.com/LSSTDESC/DC2_visitList/blob/master/DC2visitGen/notebooks/DC2_Run2_regionCoords_WFD.ipynb

| Location          | RA (degrees) | Dec (degrees) | RA (degrees) | Dec (degrees) |
|:----------------- |:------------ |:------------- |:------------ |:------------- |
| Region            | WFD          | WFD           | DDF          | DDF           |
| Center            | 61.856114    | -35.79        | 53.125       | -28.100       |
| North-East Corner | 71.462228    | -27.25        | 53.764       | -27.533       |
| North-West Corner | 52.250000    | -27.25        | 52.486       | -27.533       |
| South-West Corner | 49.917517    | -44.33        | 52.479       | -28.667       |
| South-East Corner | 73.794710    | -44.33        | 53.771       | -28.667       |

(Note that the order of the rows above is different than in the DC2 papers.  The order of the rows above goes around the perimeter in order.)

In [ ]:
dc2_run2x_wfd = [[71.462228, -27.25], [52.250000, -27.25], [49.917517, -44.33], [73.794710, -44.33]]
dc2_run2x_ddf = [[53.764, -27.533], [52.486, -27.533], [52.479, -28.667], [53.771, -28.667]]

In [ ]:
dc2_run2x_wfd_df = pd.DataFrame({'ra': [coord[0] for coord in dc2_run2x_wfd] + [dc2_run2x_wfd[0][0]],
                                 'dec': [coord[1] for coord in dc2_run2x_wfd] + [dc2_run2x_wfd[0][1]]})
dc2_run2x_ddf_df = pd.DataFrame({'ra': [coord[0] for coord in dc2_run2x_ddf] + [dc2_run2x_ddf[0][0]],
                                 'dec': [coord[1] for coord in dc2_run2x_ddf] + [dc2_run2x_ddf[0][1]]})

In [ ]:
def overlay_dc2_region(ra_dec, dc2_run2x_wfd_df=dc2_run2x_wfd_df, dc2_run2x_ddf_df=dc2_run2x_ddf_df):
    # This region isn't quite a polygon.  The sides should be curved.
    wfd_region = hv.Path(dc2_run2x_wfd_df).opts(color='red')
    ddf_region = hv.Path(dc2_run2x_ddf_df).opts(color='orange')
    ra_dec = ra_dec * wfd_region * ddf_region

    max_delta_ra = dc2_run2x_wfd_df['ra'][3] - dc2_run2x_wfd_df['ra'][2]
    delta_dec = dc2_run2x_wfd_df['dec'][1] - dc2_run2x_wfd_df['dec'][3]
    grow_buffer = 0.05

    # Notice that these are specified in increasing RA left->right
    # We rely on the invert_xaxis True above to flip this in the display
    # It's important to get this right because these ranges are used for data selection
    # and then the range is flipped in the display.
    ra_dec.opts(xlim=(dc2_run2x_wfd_df['ra'][2] - max_delta_ra * grow_buffer,
                dc2_run2x_wfd_df['ra'][3] + max_delta_ra * grow_buffer))
    ra_dec.opts(ylim=(dc2_run2x_wfd_df['dec'][3] - delta_dec * grow_buffer,
                dc2_run2x_wfd_df['dec'][1] + delta_dec * grow_buffer))

    return ra_dec

In [ ]:
def plot_ra_dec(df, dc2_run2x_wfd_df=dc2_run2x_wfd_df, dc2_run2x_ddf_df=dc2_run2x_ddf_df,
                show_dc2_region=True, cmap="bmy", bins=100, cmin=10):
    """Show rasterized RA, Dec object density.
    
    We're just doing this on a rectilinear grid
    The distortion is noticeable from the lowest to highest Dec in the change in density due to the change in area."""
    points_ra_dec = hv.Points(df, kdims=[hv.Dimension('ra', soft_range=(dc2_run2x_wfd[2][0], dc2_run2x_wfd[3][0])),
                                         hv.Dimension('dec', soft_range=(dc2_run2x_wfd[3][1], dc2_run2x_wfd[1][1]))])
    # We have to define the colormap here now, because the opts aren't passed through the datashade->Points.
    # See, e.g., https://github.com/holoviz/holoviews/issues/4125
    ra_dec = datashade(points_ra_dec, cmap=process_cmap(cmap, provider="colorcet"))
    ra_dec = ra_dec.opts(invert_xaxis=True)  # Flip to East left

    if show_dc2_region:
        ra_dec = overlay_dc2_region(ra_dec, dc2_run2x_wfd_df=dc2_run2x_wfd_df, dc2_run2x_ddf_df=dc2_run2x_ddf_df)
   
    return ra_dec

In [ ]:
ra_dec = plot_ra_dec(ra_dec_df[good])

In [ ]:
ra_dec_df.dask.dependents

In [ ]:
ra_dec_df.visualize()

This next command is the first one since the length counting above that will actually cause work to happen.  Each plot generate triggers a `compute` on the Dask DataFrame.

In [ ]:
ra_dec.opts(width=600, height=500)

The overall object density distribution looks good.

Notes:
* If you are viewing this through a direct JupyterLab connection (Jupyter Classic Notebook, or separately on your own machine or setup), the plot will re-raster as you zoom in and out.  This functionality is not available within the JupyterHub environment.  JupyterHub doesn't allow the JavaScript callbacks in the browser back to the server that are necessary to do the re-rastering.
* We explicitly excluded the tracts that overlap the DDF region (orange square upper-right corner).
* There are also a few patches that failed within the main region.
* There is an overall gradient N/S in object density, because we're plotting in rectilinear RA, Dec bins, which means that bins at the bottom in RA cover less area than those at the top.  We'll fix this next with an equal-area projection.

See the input visit coverage map here:  
https://github.com/LSSTDESC/ImageProcessingPipelines/issues/97#issuecomment-498303504


See [../contributed/ra_dec_dc2_run2.2i_dask.ipynb] for how to do an equal-area Mollweide projection.

In [ ]:
ra_dec_df.memory_usage().compute()

## Calculate effective area covered

To compare number densities of objects on the sky, we have to calculate the area covered by each catalog.
We'll use Healpix through HealPy to pixelate the region and then count of the number of pixels with significant numbers of objects.

In [ ]:
def calculate_area(df, threshold=0.25, nside=1024, verbose=False):
    """Calculate the area covered by a catalog with 'ra', 'dec'
    
    Parameters:
    --
    cat: DataFrame, dict-like with 'ra', 'dec', keys
    threshold:  float
        Fraction of median value required to count a pixel.
    nside:  int
        Healpix NSIDE.  NSIDE=1024 is ~12 sq arcmin/pixel, NSIDE=4096 is 0.74 sq. arcmin/pixel
        Increasing nside will decrease calculated area as holes become better resolved 
        and relative Poisson fluctuations in number counts become more significant.
    verbose:  bool
        Print details on nside, number of significant pixels, and area/pixel.
        
    Returns:
    --
    area:  Astropy Quantity.
    """
    import healpy as hp

    indices = hp.ang2pix(nside, df['ra'], df['dec'], lonlat=True)
    idx, counts = np.unique(indices, return_counts=True)
    
    # Take the 25% of the median value of the non-zero counts/pixel
    threshold_counts = threshold * np.median(counts)

    if verbose:
        print(f'Median {np.median(counts)} objects/pixel')
        print(f'Only count pixels with more than {threshold_counts} objects')

    significant_pixels, = np.where(counts > threshold_counts)
    area_pixel = hp.nside2pixarea(nside, degrees=True) * u.deg**2

    if verbose:
        print(f'Pixel size ~ {hp.nside2resol(nside, arcmin=True) * u.arcmin:0.2g}')
        print(f'nside: {nside}, area/pixel: {area_pixel:0.4g}, num significant pixels: {len(significant_pixels)}')

    area = len(significant_pixels) * area_pixel

    if verbose:
        print(f'Total area: {area:0.7g}')
    
    return area

In [ ]:
area_dc2 = calculate_area(ra_dec_df[galaxy])
print(f'DC2 Run 2.2i area: {area_dc2:0.2f}')

In [ ]:
area_dc2

In [ ]:
num_den_dc2 = galaxy.sum().compute() / area_dc2

# Change default expression to 1/arcmin**2
num_den_dc2 = num_den_dc2.to(1/u.arcmin**2)
print(num_den_dc2)

I now see 6.9 GB of managed memory across the cluster.

In [ ]:
ra_dec_df.dask.dependents

'del' will still leave references around if there are future computations that still might depend on `ra_dec_df`.  I believe the HoloViews objects count as something that is dependent on `ra_dec_df`.  In contrast, `client.cancel` just removes the data, whether or not anything depends on them.  So we use `client.cancel` here.  Doing this means that the data to interactively rebin RA, Dec density plot is no longer available.  We can still zoom in Bokeh, but there's no longer to recompute the binning.

In [ ]:
client.cancel(ra_dec_df)

## Color-Color Diagrams and the Stellar Locus

In [ ]:
columns = ["objectId"]
columns += [f'mag_{f}' for f in filters]
columns += [f'magerr_{f}' for f in filters]

# columns += [f'mag_{f}_cModel' for f in filters]
# columns += [f'magerr_{f}_cModel' for f in filters]
# columns += [f'I_flag']
# columns += [f'I_flag_{f}' for f in filters]
# columns += [f'Ixx_pixel_{f}' for f in filters]
# columns += [f'Ixy_pixel_{f}' for f in filters]
# columns += [f'Iyy_pixel_{f}' for f in filters]
# columns += [f'psf_fwhm_{f}' for f in filters]
# columns += ['good', 'extendedness', 'blendedness']

In [ ]:
mag_df = dd.read_parquet(
    data_path,
    columns=columns,
    engine="pyarrow",
    kwargs={"dataset": {"use_legacy_dataset": False}},
)

In [ ]:
mag_df = mag_df.set_index("objectId", sorted=True, divisions=good.divisions)

In [ ]:
# Convert mag floats to 32-bit -- that's plenty
for f in filters:
    mag_df[f"mag_{f}"] = mag_df[f"mag_{f}"].astype(np.float32)
    mag_df[f"magerr_{f}"] = mag_df[f"magerr_{f}"].astype(np.float32)

In [ ]:
# Persist to make the plots below zoomable and re-binnable.
mag_df = mag_df.persist()

Once `mag_df` is fully persisted, we are now up to 15 GB of data in the Dask cluster memory.

But I find twice that, 35 GB, in the unmanged memory.  So this implies to me some sort of memory leak or lack of reclaiming memory.

In [ ]:
# Define color columns
# These should inherit the 32-bit float type
mag_df['u-g'] = mag_df['mag_u'] - mag_df['mag_g']
mag_df['g-r'] = mag_df['mag_g'] - mag_df['mag_r']
mag_df['r-i'] = mag_df['mag_r'] - mag_df['mag_i']
mag_df['i-z'] = mag_df['mag_i'] - mag_df['mag_z']
mag_df['z-y'] = mag_df['mag_z'] - mag_df['mag_y']

In [ ]:
mag_df

In [ ]:
# We refer to a file over in `tutorials/assets' for the stellar locus
datafile_davenport = '../tutorials/assets/Davenport_2014_MNRAS_440_3430_table1.txt'

def get_stellar_locus_davenport(color1='gmr', color2='rmi',
                                datafile=datafile_davenport):
    color1 = color1.replace('-', 'm')
    color2 = color2.replace('-', 'm')

    data = pd.read_table(datafile, sep='\s+', header=1)
    return data[color1], data[color2]

    
def plot_stellar_locus(color1='gmr', color2='rmi',
                       color='blue', line_dash='dashed', line_width=2.5,
                       ax=None):

    color1_m = color1.replace('-', 'm')
    color2_m = color2.replace('-', 'm')

    model_color1, model_color2 = get_stellar_locus_davenport(color1_m, color2_m)
    model_df = pd.DataFrame({color1: model_color1, color2: model_color2})
    stellar_locus = hv.Path(model_df).opts(color='blue', line_dash=line_dash, line_width=line_width)
        
    return stellar_locus 

In [ ]:
def plot_color_color(df, color1, color2, 
                     range1=(-1, +2), range2=(-1, +2),
                     cmin=10, cmap='rainbow',
                     vmin=None, vmax=None):
    """Plot a color-color diagram.  Overlay stellar locus"""
    band1, band2 = color1[0], color1[-1]
    band3, band4 = color2[0], color2[-1]

    clean = df[np.isfinite(df[color1]) & np.isfinite(df[color2])]
    points_color1_color2 = hv.Points(
        clean,
        kdims=[
            hv.Dimension(color1, range=range1),
            hv.Dimension(color2, range=range2)]
    )

    color1_color2 = datashade(points_color1_color2, cmap=process_cmap(cmap, provider='colorcet'))

    try:
        stellar_locus = plot_stellar_locus(color1, color2)
        color1_color2 = color1_color2 * stellar_locus
    except KeyError as e:
        print(f"Couldn't plot Stellar Locus model for {color1}, {color2}")
        
    return color1_color2

In [ ]:
def plot_four_color_color(df, vmin=0, vmax=50000):
    layout = hv.Layout(
    plot_color_color(df, 'g-r', 'u-g') + \
    plot_color_color(df, 'g-r', 'r-i') + \
    plot_color_color(df, 'g-r', 'i-z') + \
    plot_color_color(df, 'g-r', 'z-y'))
    
    layout = layout.cols(2)
    
    return layout

In [ ]:
plot_four_color_color(mag_df[good])

You can see the stellar locus, and the galaxies (particularly clear in the r-i vs. g-r plot).  The Davenport model traces out a different path in color-color space than the DC2 stars -- this is likely just due to something simple like a filter conversion I (MWV) failed to do.

Note that the above panels will zoom in `g-r` together because HoloViews knows that they share this data column.  They don't zoom "together" in the y-axes because those columns are not shared between the plots.

The plots each re-raster as you zoom in and out.

There is no brushing (selection) and linking.

In [ ]:
plot_four_color_color(mag_df[star])

When we look at just the unresolved objects ("stars"), the stellar locus comes out more clearly.  The obvious galaxies are no longer visible, but there's still a significant background of unresolved objects.

The discrete islands in the data for stellar color-color plot -- most visible in `r-i` vs. `g-r` at g-r ~= 1.2 mag -- are due to the finite set of stellar models used for simulating M dwarfs.

If we restrict to looking at the bright stars, things become more clear.

The SNR requirement both does a significantly better job restricting to point sources, but also is effectively a magnitude cut as well.  And there are more stars at bright magnitudes than fainter.

In [ ]:
bright_magerr_threshold = 0.05
bright_star = star & (mag_df["magerr_r"] < bright_magerr_threshold)

In [ ]:
plot_four_color_color(mag_df[bright_star])

------
Let's plot the galaxies on the same color-color plots

Clearly one doesn't expect the galaxies to follow the stellar locus.  But including the stellar locus lines makes it easy to guide the eye between the stars-only and the galaxies-only plots.  

In [ ]:
plot_four_color_color(mag_df[galaxy])

The galaxies that appear to follow the stellar locus in r-i vs. g-r are the red sequence galaxies.

Questions for further study:
   1. Is there a better comparison sample for the stellar locus than the Davenport reference?
   2. Why is the stellar locus in the Davenport 0.1--0.2 mag redder for the reddest stars than the observed data.  Are there different extinction assumptions (this should be a low-extinction region).  Are there different bandpasses used?

The Dask cluster is up 18 GB in managed memory, and another 18 GB in unmanaged memory.

## 1D Density Plots

In [ ]:
def plot_data_curve(data, bins, norm=None, line_color=None, line_dash=None):
    """
    Parameters
    ----------
    data : np.array like
    bins : array of bins
    norm : Normalize frequencies by this factor.  E.g., divide by area to get counts/area.
    line_color : Valid cmap for hv.Curve(line_color=)
    line_dash : Valid line dash style for hv.Curve(line_dash=)
    """
    frequencies, edges = da.histogram(data, bins=bins)
    if norm is not None:
        frequencies /= norm

    centers = (edges[:-1] + edges[1:]) / 2
    step_edges = da.dstack((edges[:-1], edges[1:])).flatten()
    step_frequencies = da.dstack((frequencies, frequencies)).flatten()
    
    curve = hv.Curve((centers, frequencies)).opts(interpolation='steps-mid')

    if line_color is not None:
        curve.opts(line_color=line_color)
    if line_dash is not None:
        curve.opts(line_dash=line_dash)

    return curve

In [ ]:
def plot_density_curve(df, data_col,
                       bins=None, area=None, **kwargs):
    """
    area: in square arcminutes
    """
    curve = plot_data_curve(df[data_col], bins=bins, norm=area, **kwargs)

    # Assume equally bin sizes
    bin_size = bins[1] - bins[0]
    if area is not None:
        ylabel = f"Objects/square arcmin/{bin_size:0.2} mag bin"
    else:
        ylabel = "Objects/bin"
    
    curve.opts(xlabel=data_col, ylabel=ylabel)

    return curve 


def plot_mag_densities(df, good, star, galaxy, filt,
                       area=None,
                       log=False, range=(16, 32), bins=None,
                       legend_position='top_left'):
    if bins is None:
        bins = np.linspace(*range, 160)
    
    data_col = f'mag_{filt}'
    densities = {}
    for idx, name, color in zip((good, star, galaxy), ('good', 'star', 'galaxy'), ('green', 'blue', 'red')):
        curve = plot_density_curve(df[idx], data_col, bins=bins, area=area, line_color=color)
        densities[name] = curve

    overlay = hv.NdOverlay(densities, kdims='Object Type')
    overlay.group = filt
    overlay.opts(show_legend=True, legend_position=legend_position)
    if log:
        overlay.opts(logy=True)
    
    return overlay


In [ ]:
density_plots = [
    plot_mag_densities(mag_df, good, star, galaxy, filt, area=area_dc2.to_value(u.arcmin**2))
    for filt in filters
]

In [ ]:
for m in density_plots[1:]:
    m.opts(show_legend=False)

In [ ]:
mag_densities = hv.Layout(density_plots)

In [ ]:
mag_densities.cols(3)

The sharp cut in i-band is because that was the reference band for most detections.  The distributions in the other bands extend to 28th mag because many of the forced-photometry measurements are consistent with 0 and our S/N cut above was on i-band flux.

Note:
* The above set of plots zoom together in plot range.  This is not based in any of the data values, just the plots being displayed and zooming together in the numerical value of the x axis.

## Magnitude Error vs. Magnitude

The magnitude uncertainties come directly from the poisson estimates of the flux measurements.  By construction they will follow smooth curves.  We here confirm that they do.

In [ ]:
def plot_mag_magerr(df, band, ax=None, range=(16, 28), magerr_limit=0.25, vmin=100,
                   cmap="rainbow", snr_magerr_threshold=magerr_cut):
    mag_col, magerr_col = f'mag_{band}', f'magerr_{band}'
    points_mag_magerr = hv.Points(df, kdims=[hv.Dimension(mag_col, range=(14, 28)),
                                             hv.Dimension(magerr_col, range=(0, snr_magerr_threshold))])
    return datashade(points_mag_magerr, cmap=process_cmap(cmap, provider='colorcet'))


In [ ]:
mag_magerr = hv.Layout([plot_mag_magerr(mag_df[good], filt) for filt in filters])

The following line that actually computes and realizes the graphs fails for me the first time with an "`int` not iterable" error in Bokeh, but then works when I re-run it.  I suspect some default is not getting set properly; likely a bug.

In [ ]:
mag_magerr.cols(3)

Memory usage is now at 19 GB managed with another 17 GB unmanaged.

We're next going to focus on some other columns now.  To keep memory usage lower, we will delete the `mag_df` and reload it with just the columns we need.

If we use `client.cancel` instead of `del` to delete mag_df we get down to 8.5 GB managed.

'del' will still leave references around if there are future computations that still might depend on `mag_df`.  I believe the HoloViews objects count as something that is dependent on `mag_df`.  In contrast, `client.cancel` just removes the data, whether or not anything depends on them.

*However* this will make the `datashader` plots above no longer work.  And, if they weren't on screen when generated, they will never appear in the Bokeh rendering.  So if you've been running this Notebook cell by cell then things are fine.  But if you ran this Notebook in one go you will be surprised that the Points plots will be blank.

In [ ]:
client.cancel(mag_df)

## Blendedness, Extendedness, and mag_cModel

In [ ]:
columns = ["objectId"]
columns += ["mag_g", "mag_r", "mag_i"]
columns += ["blendedness", "extendedness"]
columns += [f"mag_{f}_cModel" for f in filters]
columns += [f"magerr_{f}_cModel" for f in filters]

In [ ]:
# columns += [f'mag_{f}_cModel' for f in filters]
# columns += [f'magerr_{f}_cModel' for f in filters]
# columns += [f'I_flag']
# columns += [f'I_flag_{f}' for f in filters]
# columns += [f'Ixx_pixel_{f}' for f in filters]
# columns += [f'Ixy_pixel_{f}' for f in filters]
# columns += [f'Iyy_pixel_{f}' for f in filters]
# columns += [f'psf_fwhm_{f}' for f in filters]
# columns += ['good', 'extendedness', 'blendedness']

In [ ]:
extended_df = dd.read_parquet(
    data_path,
    columns=columns,
    engine="pyarrow",
    kwargs={"dataset": {"use_legacy_dataset": False}},
)

In [ ]:
extended_df = extended_df.set_index("objectId", sorted=True, divisions=good.divisions)

In [ ]:
# Convert mag floats to 32-bit -- that's plenty
for col in extended_df.columns:
    extended_df[col] = extended_df[col].astype(np.float32)

We're carrying a few more columns around so the following is going to use up memory, but both (1) the first run performance and (2) the interactive performance are better if we do it now.

1. To be specific, while a column-based store like Parquet is indeed great at reading just one column, it's still faster to read two columns at a time when going through the files than to read one column, and then read a second column in another pass.
2. The interactive usage makes repeated calls to fetch the data.  This gets really slow.

In [ ]:
extended_df = extended_df.persist()

This takes ~ 10 minutes goes up to 40 GB of memory.  The dominant users are the `assign` and `transfer|group|split`-`shuffle`s.  I want to investigate this further.

But after this is all done, we get back to just:

Memory usage: 6.5 GB managed.  17.8 unmanaged.   14.8 GB spilled to disk.

We put this trivial color calculation after the persist:

In [ ]:
# Define color columns
# These should inherit the 32-bit float type
# extended_df['u-g'] = extended_df['mag_u'] - extended_df['mag_g']
extended_df['g-r'] = extended_df['mag_g'] - extended_df['mag_r']
# extended_df['r-i'] = extended_df['mag_r'] - extended_df['mag_i']
# extended_df['i-z'] = extended_df['mag_i'] - extended_df['mag_z']
# extended_df['z-y'] = extended_df['mag_z'] - extended_df['mag_y']

## Blendedness

Blendedness is a measure of how much the identified flux from an object is affected by overlapping from other objects.

See Bosch et al., 2018, Section 4.9.11.

Questions for futher study:
1. What do negative blendedness scores mean?
2. What happened to yield non-finite blendedness measurements?

In [ ]:
blendedness = datashade(
    hv.Points(extended_df[good], kdims=["mag_i", "blendedness"]),
    cmap=process_cmap("rainbow", provider="colorcet"),
)

In [ ]:
blendedness.opts(xlim=(15, 26.5), ylim=(-0.5, 1))

### Extendedness
 
Extendedness is essentially star/galaxy separation based purely on morphology in the main detected reference band (which is `i` for most Objects).

Extendedness a binary property in the catalog, so it's either 0 or 1.

In [ ]:
extendedness = datashade(
    hv.Points(extended_df[good], kdims=["mag_i", "extendedness"]),
    cmap=process_cmap("rainbow", provider="colorcet"),
)

In [ ]:
extendedness.opts(ylim=(-0.1, +1.1)) * hv.Text(18, 0.9, "Galaxies") * hv.Text(18, 0.1, "Stars")

Extendedness is based off of the difference between the point-source model and extended model brightness.  Specifically objects with `mag_psf - mag_cmodel > 0.164` mag are labeled with `extendedness=1` (i.e., galaxies).

As galaxies get smaller in angular size and lower in signal-to-noise ratio, it becomes harder to clearly distinguish stars from galaxies.

See Bosch et al. 2018, Section 4.9.10 for details.

In [ ]:
extendedness_delta_mag_cut = 0.0164
psf_cModel_mag_cut = hv.VLine(
    extendedness_delta_mag_cut,
    label=rf"{extendedness_delta_mag_cut:0.4f} $\Delta$mag cut",
)
psf_cModel_mag_cut = psf_cModel_mag_cut.opts(color="green", line_dash="dashed")

In [ ]:
def plot_delta_mag_cModel(df, filt, bins=None):
    if bins is None:
        bins = np.linspace(-0.1, 0.1, 201)
    frequencies, edges = da.histogram(
        df[f"mag_{filt}"] - df[f"mag_{filt}_cModel"], bins=bins
    )
    return hv.Curve((edges[:-1], frequencies)).opts(interpolation="steps-pre")

In [ ]:
filt = "i"
delta_mag_cModel_hists = {
    "good": plot_delta_mag_cModel(extended_df[good], filt),
    "star": plot_delta_mag_cModel(extended_df[star], filt),
    "galaxy": plot_delta_mag_cModel(extended_df[galaxy], filt),
}

In [ ]:
delta_mag_cModel = hv.NdOverlay(delta_mag_cModel_hists, kdims="Sample") 

In [ ]:
delta_mag_cModel.opts(width=600, xlabel='mag_i[_psf] - mag_i_CModel', ylabel='Objects/bin') \
  * psf_cModel_mag_cut \
  * hv.Text(-0.05, 4000, "Stars") * hv.Text(0.05, 4000, "Galaxies")

Objects that are noticeably brighter in mag_i_CModel than mag_i (which is PSF mag) have clearly more flux going out and are interpreted as galaxies.  The classification of extendedness 1: "galaxy" vs. extendedness 0: "star" is based on this difference.

In [ ]:
extended_df["delta_mag_cModel_i"] = extended_df["mag_i"] - extended_df["mag_i_cModel"]
clean = (
    good 
    & (-2.5 < extended_df["g-r"])
    & (extended_df["g-r"] < 4)
    & da.isfinite(extended_df["delta_mag_cModel_i"])
)

In [ ]:
psf_cModel_mag_cut = hv.HLine(extendedness_delta_mag_cut,
                              label=rf"{extendedness_delta_mag_cut:0.4f} $\Delta$mag cut")
psf_cModel_mag_cut = psf_cModel_mag_cut.opts(color='green', line_dash="dashed")

In [ ]:
points = hv.Points(extended_df[clean], kdims=['mag_i', 'delta_mag_cModel_i'])
points = points.opts(xlabel='mag_i[_psf] - mag_cModel_i')

yhist = points.hist(dimension='delta_mag_cModel_i', adjoin=False)
xhist = points.hist(dimension='mag_i', adjoin=False)

shaded_points = datashade(points, cmap=process_cmap("rainbow", provider="colorcet"))

In [ ]:
points_color = hv.Points(extended_df[clean], kdims=['g-r', 'delta_mag_cModel_i'])
points_color_xhist = points_color.hist(dimension='g-r', dynamic=True, adjoin=False)

shaded_points_color = datashade(points_color, cmap=process_cmap("rainbow", provider="colorcet"))

In [ ]:
composite = (shaded_points_color << yhist << points_color_xhist) \
    + (shaded_points * psf_cModel_mag_cut << yhist << xhist)

In [ ]:
composite

We can zoom in a little to see how the fixed 0.0164 mag cut works at the low SNR limit.  Specifically at mag 24, we're starting to run out of stars and most things are galaxies.  But that's a population prior, it's not something visible using just morphology information.

You can see the effect of lower SNR measurements as the horizontal line at $\Delta$mag=0 puff up due to increased uncertainties.

TODO: 
1. I don't know how to construct an AdjointLayout without a "right" element.  So there's an extra duplicate "delta_mag_cModel_i" histogram that's not really helpful or projected correctly.

Now we drop this dataframe to free up memory on the Dask cluster for analyses of other columns

In [ ]:
client.cancel(extended_df)

This drops us down to 6.5 GB.  There's 22 GB unmanaged and 4 GB spilled to disk.

## Shape Parameters

Ixx, Iyy, Ixy

In [ ]:
columns = ["objectId"]

columns += [f"Ixx_pixel_{f}" for f in filters]
columns += [f"Ixy_pixel_{f}" for f in filters]
columns += [f"Iyy_pixel_{f}" for f in filters]

In [ ]:
moment_df = dd.read_parquet(
    data_path,
    columns=columns,
    engine="pyarrow",
    kwargs={"dataset": {"use_legacy_dataset": False}},
)

In [ ]:
moment_df = moment_df.set_index("objectId", sorted=True, divisions=good.divisions)

In [ ]:
for col in moment_df.columns:
    moment_df[col] = moment_df[col].astype(np.float32)

In [ ]:
moment_df = moment_df.persist()

In [ ]:
def plot_moments_for_filter(df, good, star, galaxy, filt,
                            names=['good', 'star', 'galaxy'],
                            colors=['blue', 'orange', 'green']):
    curve_kwargs = {'color': colors, 'log': True,
             'range': (0, 50)}

    bins = np.logspace(-1, 1.5, 100)
    moment_lines = {}
    for prefix, ls in (('Ixx', 'solid'), ('Iyy', 'dashed'), ('Ixy', 'dotted')):
        field = f'{prefix}_pixel_{filt}'
        for idx, name, color in zip((good, star, galaxy), names, colors):
            label = f'{prefix} {name}'
            line = plot_data_curve(df.loc[idx, field], bins=bins, line_color=color, line_dash=ls)
            moment_lines[label] = line

    moments_plot = hv.NdOverlay(moment_lines, kdims="Moments")
    moments_plot.opts(xlabel=f'{filt} Moments: Ixx, Iyy, Ixy [pixels^2]')
    moments_plot.opts(ylabel='objects / bin')
    
    return moments_plot

In [ ]:
moment_df

In [ ]:
moment_plots = [plot_moments_for_filter(moment_df, good, star, galaxy, filt) for filt in filters]

In [ ]:
### Only include legend on the first plot
for m in moment_plots[1:]:
    m.opts(show_legend=False)

ymin = 100
for m in moment_plots:
    m.opts(logy=True, ylim=(ymin, None))

moments = hv.Layout(moment_plots)

In [ ]:
moments.cols(3)

TODO:
1. Shift to a legend that's outside the grid of plots

The stars (orange) are concentrated at low values of the source moments.

Would be interesting to
1. Look by magnitude or SNR to understand the longer tail.  Are these galaxies mis-classified as stars, or are these noise sources?
2. Distribution of ellipticity (see validate_drp to type this right)

## Ellipticity

### Define ellipticity.

First we compute the ellipticities.  This will create 18 more columns, so we didn't generate this until we needed it now.  We will have to re-define `star` and `galaxy` if we want those arrays to have these new columns.

In [ ]:
def ellipticity_e_e1_e2(df, Ixx, Ixy, Iyy):
    """Calculate ellipticity from second moments from a dataframe.

    Returns e, e1, and e2.  Note that e1 and e2 are just the real and imaginary parts of e.

    Parameters
    ----------
    df : DataFrame
    Ixx : column name
    Ixy : column name
    Iyy : column name

    Returns
    -------
    e, e1, e2 : (float, float, float) or (numpy.array, numpy.array, numpy.array)
        Complex ellipticity, real component, imaginary component
        
    Copied from https://github.com/lsst/validate_drp/python/lsst/validate/drp/util.py
    """
    e = (df[Ixx] - df[Iyy] + 2j*df[Ixy] ) / (df[Ixx] + df[Iyy] + 2*da.sqrt(df[Ixx]*df[Iyy] - df[Ixy]**2))
    e1 = da.real(e)
    e2 = da.imag(e)
    return e, e1, e2

In [ ]:
def ellipticity(df, Ixx, Ixy, Iyy):
    """Calculate ellipticity from second moments from a dataframe.

    Just returns the complex e.

    Parameters
    ----------
    df : DataFrame
    Ixx : column name
    Ixy : column name
    Iyy : column name

    Returns
    -------
    e, e1, e2 : (float, float, float) or (numpy.array, numpy.array, numpy.array)
        Complex ellipticity, real component, imaginary component
        
    Copied from https://github.com/lsst/validate_drp/python/lsst/validate/drp/util.py
    """
    e = (df[Ixx] - df[Iyy] + 2j*df[Ixy] ) / (df[Ixx] + df[Iyy] + 2*da.sqrt(df[Ixx]*df[Iyy] - df[Ixy]**2))
 
    return e

In [ ]:
def make_ellipticity_df(df):
    for filt in filters:
        Ixx, Ixy, Iyy = f"Ixx_pixel_{filt}", f"Ixy_{filt}", f"Iyy_{filt}"
        e, e1, e2 = ellipticity(df, Ixx, Ixy, Iyy)

        # 1. Is this really the right way to do this?
        # 2. Is this an efficient way to do this?
        df[f"e_{filt}"] = e
        df[f"e1_{filt}"] = e1
        df[f"e2_{filt}"] = e2
        
    return df

In [ ]:
def calc_ellipticity_df(df):
    for filt in filters:
        Ixx, Ixy, Iyy = f"Ixx_pixel_{filt}", f"Ixy_pixel_{filt}", f"Iyy_pixel_{filt}"
        df[f"e_{filt}"] = ellipticity(df, Ixx, Ixy, Iyy)

        # 1. Is this really the right way to do this?
        # 2. Is this an efficient way to do this?
#        e_df[f"e_{filt}"] = e
#        e_df[f"e1_{filt}"] = e1
#        e_df[f"e2_{filt}"] = e2
        
    return df

In [ ]:
moment_df = calc_ellipticity_df(moment_df[good])

Persist because we're going to use these in a couple of different ways below.
This `persist` triggers the ellipticity calculation above to compute and the subsequent merge with `good`.
This takes about 10-15 seconds on 2 NERSC Haswell nodes.

In [ ]:
moment_df = moment_df.persist()

In [ ]:
def plot_ellipticities_for_filter(df, good, star, galaxy, filt,
                                names=['good', 'star', 'galaxy'],
                                colors=['blue', 'orange', 'green']):
    hist_kwargs = {'color': colors, 'log': True, 'range': (0, 50)}

    bins = np.linspace(0, 1, 51)
    ellipticity_lines = {}
    for idx, name, color in zip((good, star, galaxy), names, colors):
        field = f'e_{filt}'

        label = f'e {name}'
        line = plot_data_curve(df.loc[idx, field], bins=bins, line_color=color, line_dash='solid')
        ellipticity_lines[label] = line
        
        label = f'e1 {name}'
        line = plot_data_curve(da.real(df.loc[idx, field]), bins=bins, line_color=color, line_dash='dashed')
        ellipticity_lines[label] = line

        label = f'e2 {name}'
        line = plot_data_curve(da.imag(df.loc[idx, field]), bins=bins, line_color=color, line_dash='dotted')
        ellipticity_lines[label] = line

    ellipticities_plot = hv.NdOverlay(ellipticity_lines, kdims="Ellipticities")
    ellipticities_plot.opts(xlabel=f'{filt} ellipticity: e, e1, e2 [pixels^2]')
    ellipticities_plot.opts(ylabel='objects / bin')

    return ellipticities_plot    

In [ ]:
ellipticity_plots = [plot_ellipticities_for_filter(moment_df, good, star, galaxy, filt) for filt in filters]

In [ ]:
### Only include legend on the first plot
for m in ellipticity_plots[1:]:
    m.opts(show_legend=False)

ellipticities = hv.Layout(ellipticity_plots)

In [ ]:
ellipticities.opts(hv.opts.Curve(xlim=(0, 1), ylim=(100, None), logy=True)).cols(3)

In [ ]:
client.cancel(moment_df)

### FWHM of the PSF
At the location of the catalog objects.

The Object Table stores the shape parameters of the PSF model as evaluated at the location of the object.

This is not the same as, but is certainly related to, the distribution of effective seeing in the individual images that made up the coadd.

In [ ]:
columns = ["objectId"]

columns += [f"psf_fwhm_{f}" for f in filters]

In [ ]:
psf_df = dd.read_parquet(
    data_path,
    columns=columns,
    engine="pyarrow",
    kwargs={"dataset": {"use_legacy_dataset": False}},
)

In [ ]:
psf_df = psf_df.set_index("objectId", sorted=True, divisions=good.divisions)

In [ ]:
for col in psf_df.columns:
    psf_df[col] = psf_df[col].astype(np.float32)

In [ ]:
psf_df = psf_df.persist()

In [ ]:
def plot_psf_fwhm(df, filt, bins=None, density=True):
    psf_col = f"psf_fwhm_{filt}"
    curve = plot_data_curve(df[psf_col], bins=bins, norm=len(df))
    curve.opts(xlabel=psf_col)
    return curve


def plot_psf_fwhm_for_filters(
    df,
    filters=filters,
    bins=None,
    density=True,
    xlim=(0.5, 1.4),
    alpha=0.5,
    colors=("purple", "blue", "green", "orange", "red", "brown"),
):
    if bins is None:
        bins = np.linspace(0, 1.5, 201)

    fwhm_histograms = {}
    for filt, color in zip(filters, colors):
        curve = plot_psf_fwhm(df, filt, bins=bins, density=density)
        curve.opts(line_color=color)
        fwhm_histograms[filt] = curve

    fwhm = hv.NdOverlay(fwhm_histograms, kdims="Filter")
    if density:
        ylabel = "Density [Normalize to sum=1]"
    else:
        ylabel = "Objects / bin"
    fwhm.opts(xlabel="Model PSF FWHM [arcsec]")
    fwhm.opts(ylabel=ylabel)
    fwhm.opts(xlim=xlim)

    return fwhm

In [ ]:
plot_psf_fwhm_for_filters(psf_df[good]).opts(width=600, xlim=(0.5, 1.4))

The y-band observations have a wider PSF FWHM because we didn't focus the simulated telescope properly in the simulated images.  The telescope focus was set to the surface of the silicon.  But the conversion happens deeper, at increasing depth with increasing wavelength.  In addition, we did not correctly refract the photons in the silicon of the CCD, so the effective focal plane was even deeper in the silicon than it should have been.

In [ ]:
client.cancel(psf_df)